In [1]:
from pathlib import Path
import timm
import torch
import pandas as pd
from PIL import Image
import numpy as np
import pyrootutils

root = pyrootutils.setup_root(
    search_from=Path.cwd().parent,
    indicator=".project-root",
    pythonpath=True,
    dotenv=True,
)

from src.data_modules.schisto import SchistoDataModule

/home/kris/miniforge3/envs/flim/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
image_size = 512
datamodule = SchistoDataModule(
    batch_size=1,
    use_flim_data=False,
    # transforms={
    #     "train": [ 
    #         {
    #             "class_path": "Resize",
    #             "init_args": {
    #                 "height": image_size,
    #                 "width": image_size
    #             }
    #         }
    #     ]
    #             },
    num_workers=8)
datamodule.prepare_data()
datamodule.setup()
train_dataloader = datamodule.train_dataloader()

print(f"Number of training batches: {len(train_dataloader)}")

Dataset summary:
  FLIM train images: 4
  GWE train images: 163
  GWE val images: 163
  Test images: 610
Number of training batches: 163


In [3]:
item = next(iter(train_dataloader))
images = item["image"]
label = item["label"]

print(f"Image batch shape: {images.shape}")
print(f"Label batch shape: {label.shape}")

Image batch shape: torch.Size([1, 3, 512, 512])
Label batch shape: torch.Size([1, 512, 512])


In [4]:
model = timm.create_model('vit_base_patch16_dinov3_qkvb.lvd1689m', 
                          pretrained=True,
                          features_only=True,
                          )

In [5]:
features_list = model(images)

In [7]:
# reshape feature from (batch_size, num_channels, patch_h, patch_w) to (batch_size, num_channels, patch_h * patch_w)
features_list[-1].shape

# plot the features as images
import matplotlib.pyplot as plt
num_features = features_list[-1].shape[1]
fig, axes = plt.subplots(1, num_features, figsize=(20, 5))
for i in range(num_features):
    feature_map = features_list[-1][0, i, :, :].detach().cpu().numpy()
    axes[i].imshow(feature_map, cmap='viridis')
    axes[i].axis('off')
plt.show()